In [12]:
'''
implement keeloq decryption but with printing out a specific round
use numpy to perform correlation with power trace
'''
# base on https://eprint.iacr.org/2008/058.pdf 
# Physical Cryptanalysis of KeeLoq Code Hopping Applications

import numpy as np
import numpy.linalg as la
import datetime
import struct
import os 
import csv
from time import time
import random
import math
from operator import itemgetter
import pandas as pd 


In [2]:
KeeLoq_NLF=0x3A5C742E
 
def bit(x,n):
    ''' Get the n'th bit from x'''
    y=(((x)>>(n))&1)
    return y
    
def g5(x,a,b,c,d,e):
    ''' Non linear mixing of bits'''
    y=(bit(x,a)+bit(x,b)*2+bit(x,c)*4+bit(x,d)*8+bit(x,e)*16)
    return y


def swap_device_key_decrypt_key(device_key): 
    #take 64 and then make sure the first bit is the last bit used in decrypting 
    return(device_key[-17::-1] + device_key[:-17:-1])
        
def key_register(key):
    #take a base 10 digit convert it into the actually key in the right format (flipped and left rotated 16 bits)
    key = bin(key)[2:] #get rid of 0b
    key = key.zfill(64)#64 chars
    key = key[-17::-1] + key[:-17:-1]
    key = int(key,2)   #convert to binary
    return key
  

def bin_rep(x, width = 16):
    ''' binary representation'''
    return(bin(x)[2:].zfill(width))


def keeloq_encrypt(plain,key):
    temp = plain
    for r in range(528) :
        temp = (temp>>1) ^ ( (bit(temp,0)^bit(temp,16)^bit(key,r&63)^bit(KeeLoq_NLF,g5(temp,1,9,20,26,31)))<<31)
    return(temp&0xFFFFFFFF)

def decrypt_round(cipher,device_key,decrypt_round):
    '''one round of a keeloq decryption'''
    #The key starts at bit 15 and then gets left shifted (next bit used is 14)
    return ((cipher<<1) ^bit(cipher,31) ^bit(cipher,15) ^bit(device_key,(15-decrypt_round)&63) ^bit(KeeLoq_NLF,g5(cipher,0,8,19,25,30))) & 0xFFFFFFFF
        
def partial_decrypt (cipher,device_key,stop):
    '''do a partial decryption'''
    for r in range(stop):
        cipher = decrypt_round(cipher,device_key,r)
    
    return cipher&0xFFFFFFFF #y0

def resume_decrypt (cipher,device_key,start,stop):
    '''start in the middle of a decrypt and stop whenever'''
    for r in range(start,stop):
        cipher = decrypt_round(cipher,device_key,r)
    
    return cipher&0xFFFFFFFF #y0

def keeloq_decrypt(cipher,device_key):
    '''A full decrypt is 528 rounds'''
    return partial_decrypt(cipher,device_key,528)

def partial_decrypt_power(cipher,device_key,stop):
    ''' Get the power usage after a partial decrypt'''
    for r in range(stop):
        if r == stop - 1 : y1 = cipher&0xFFFFFFFF
        cipher = decrypt_round(cipher,device_key,r)
        if r == stop - 1 : y0 = cipher&0xFFFFFFFF
        
    #the power output is the hamming distance between the last two rounds    
    power = bin(y1^y0).count('1')

    return power
        
def partial_decrypt_power_array(cipher,device_key,stop):
    ''' Get the history of power usage when decrypting '''
    power = []
    prev = cipher
    try:
        for r in range(stop):
            cipher = decrypt_round(cipher,device_key,r)
            power.append((bin(prev^cipher)).count('1')) #hamming distance between this and previous round
            prev = cipher

        return power[::-1]        
    except Exception as e:
        print(cipher,type(cipher),device_key,type(device_key),e)
        
def partial_decrypt_power_array_with_noise(cipher,device_key,stop):
    p = partial_decrypt_power_array(cipher,device_key,stop)
    power = [random.gauss(phyp,0.01*phyp) for phyp in p ]
    #random guassian noise is added to simulate reality, although we dont know what kind of noise exists in reality, most likely variance it is proportional to peak power 
    return power



In [59]:
def read_cipher():
        ciphertext=[]
        ciphertextfile=open(r"./25C3_HCS301/ciphertexts.txt","r")
        for line in ciphertextfile:
            ciphertext.append(int(line,16))
        #print(ciphertext[888])
        #line 889 index starts at 0
        ciphertextfile.close()
        return ciphertext
        
def read_powertrace():
    Peak=[]
    for i in range(997):
        Peak.append(i)
        Peak[i]=[]
        with open(f"./25C3_HCS301/Peak{i+1}.dat",mode='rb') as file:
            for j in range(1650):
                byte = file.read(8)
                Peak[i].append(struct.unpack('d',byte)[0])
        file.close()
    return Peak


def read_powertrace(ciphertexts):
    ciphertexts_d = {}
    for i,cipher in enumerate(ciphertexts):
        ciphertexts_d[cipher] = {}
        file_path = f"./25C3_HCS301/Peak{i+1}.dat" 
        #print(os.path.getsize(file_path))
        with open(file_path,mode='rb') as file:
            peak_data = np.fromfile(file, dtype=np.float64, count=1650)
            ciphertexts_d[cipher]['powertrace_sim'] = peak_data
        file.close()
    return ciphertexts_d


#dummy_ciphertexts = read_cipher()
#dummy_ciphertexts

#ciphertexts_d = read_powertrace(dummy_ciphertexts)
#ciphertexts_d
#print(ciphertexts_d[dummy_ciphertexts[0]]['powertrace_sim'][0:20])
#print(len([x for x in ciphertexts_d[dummy_ciphertexts[8]]['powertrace_sim'] if x >2]  ))
#typically 545 readings over 2, means more than 528 rounds, must try more rounds

#ciphertexts_d[dummy_ciphertexts[8]]['powertrace_sim']

def avg3(array):

    test = array
    reshaped_array = test.reshape(-1, 3) 
    averages = reshaped_array.mean(axis=1)
    return(averages)

for k,v in ciphertexts_d.items():
    v['powertrace_sim_avg'] = avg3(v['powertrace_sim'])
 


In [16]:
df = pd.DataFrame.from_dict(
    {cipher: data['powertrace_sim'] for cipher, data in ciphertexts_d.items()},
    orient='index'
)

# Save the DataFrame to a CSV file
csv_file_path = './powertraces.csv'
df.to_csv(csv_file_path, header=True)

print(f"Powertraces saved to {csv_file_path}")

Powertraces saved to powertraces.csv


In [76]:
'''
Input: m : length of key guess, n: number of surviving key guesses, k: known previous key bits
Output: SurvivingKeys
    1: KeyHyp = {0, 1}^m
    2: for all KeyHypi; 0 ≤ i < 2^m do
        3: Perform CPA on round (528 − m) using PHyp and k
    4: end for
    5: SurvivingKeys = n most probable partial keys of KeyHyp
'''

def algorithm1(key_length_guess, n, ciphertexts , power_traces , prev_keys = [0] , current_bits_guessed = 0 ):
    '''
    guess m bits of the keys (try 2**m) keys, take the best n
    best n is based on the higest correlation between the actual power and the power hypothesis based on the guessed keys 
    '''
    
    #put into normal form
    temp_1 = [ key_register(x) for x in  prev_keys]  
    #add more MSB for guessing 
    temp_2 = [  ( (y <<current_bits_guessed) ^ x) for x in temp_1 for y in range(2**key_length_guess)]
    #put back into key register form for decrypting
    keyhyp = [ key_register(x) for x in  temp_2 ]
    #all combinations of test keys and ciphertexts
    keyhyp_d = { (key, ciphertext) : {} for key in keyhyp for ciphertext in ciphertexts}
    
    key_depth = current_bits_guessed + key_length_guess
    
    for key,v in keyhyp_d.items():
        device_keyhyp = key[0]
        ciphertext = key[1]
        
        v['test_key'] = device_keyhyp  
        v['bin_repr'] = bin(device_keyhyp)[2:].zfill(64)  
        v['dummy_power'] = partial_decrypt_power_array(ciphertext, device_keyhyp ,key_depth)[0]
        v['power_trace'] = power_traces[ciphertext]['powertrace_sim_avg'][-key_depth]

    df = pd.DataFrame([{'test_key':k[0], 'ciphertext':k[1], 'bin_repr': v['bin_repr'], 'dummy_power': v['dummy_power'], 'power_trace': v['power_trace']} for k,v in keyhyp_d.items()])
    #display(t)

    correlation_by_group = df.groupby(['test_key', 'bin_repr']).apply(lambda g: g['dummy_power'].corr(g['power_trace'])).reset_index()
    sorted_corr = correlation_by_group.sort_values(by=0,ascending=False)
    print(sorted_corr[:5]) 
    
    return({"likely_keys" : list(sorted_corr['test_key'][0:n]), "current_bits_guessed" : key_depth }) #Get the four most likely( highest max correlation) key guesses

#t1 = algorithm1(8, 10, dummy_ciphertexts[0:500] , ciphertexts_d, [0] , 0   )
#t2 = algorithm1(8, 10, dummy_ciphertexts[0:500] , ciphertexts_d, t1["likely_keys"] , t1["current_bits_guessed"]   )
#t3 = algorithm1(4, 10, dummy_ciphertexts[0:500] , ciphertexts_d, t2["likely_keys"][0:5] , t2["current_bits_guessed"]   )
#t4 = algorithm1(4, 10, dummy_ciphertexts[0:500] , ciphertexts_d, t3["likely_keys"][0:5] , t3["current_bits_guessed"]   )


#t1 = algorithm1(8, 10, dummy_ciphertexts[0:500]  , ciphertexts_d, [151050434977488319          ] , 56   )


for offset in range(1,17):
    for start in range(3):
        start_time =  datetime.datetime.now()
        print(f"start:{start}, offset:{offset}")

        for k,v in ciphertexts_d.items():
            v['powertrace_sim_avg'] = (v['powertrace_sim'])[start::3][:-offset]
        t1 = algorithm1(8, 10, dummy_ciphertexts , ciphertexts_d, [0] , 0   )
        print(datetime.datetime.now() - start_time)

#avg3 offeset 3, 0.55
#offset 3 is the best

start:0, offset:1
     test_key                                           bin_repr         0
39       9984  0000000000000000000000000000000000000000000000...  0.041809
199     50944  0000000000000000000000000000000000000000000000...  0.035865
231     59136  0000000000000000000000000000000000000000000000...  0.034491
38       9728  0000000000000000000000000000000000000000000000...  0.034328
71      18176  0000000000000000000000000000000000000000000000...  0.033293
0:00:12.763446
start:1, offset:1
     test_key                                           bin_repr         0
68      17408  0000000000000000000000000000000000000000000000...  0.432013
69      17664  0000000000000000000000000000000000000000000000...  0.428917
71      18176  0000000000000000000000000000000000000000000000...  0.426458
70      17920  0000000000000000000000000000000000000000000000...  0.423314
196     50176  0000000000000000000000000000000000000000000000...  0.391622
0:00:10.465195
start:2, offset:1
    test_key    

In [81]:
start_time =  datetime.datetime.now()
bits = 4 

#v['powertrace_sim_avg'] = (v['powertrace_sim'])[2::3][:-3]
#v['powertrace_sim_avg'] = avg3(v['powertrace_sim'])[:-2]

for k,v in ciphertexts_d.items():
    v['powertrace_sim_avg'] = avg3(v['powertrace_sim'])[:-3]

results = {"likely_keys" : [0], "current_bits_guessed" : 0 }
    
for i in range(64//bits):
    inner_start_time =  datetime.datetime.now()
    results = algorithm1(4, 10, dummy_ciphertexts , ciphertexts_d, results["likely_keys"] , results["current_bits_guessed"]   )
    print(datetime.datetime.now() - inner_start_time)
    #print([bin(x)[2:].zfill(64)   for x in results["likely_keys"]])
    
print(datetime.datetime.now() - start_time)
print(results)
    


    test_key                                           bin_repr         0
4      16384  0000000000000000000000000000000000000000000000...  0.621136
12     49152  0000000000000000000000000000000000000000000000...  0.553974
8      32768  0000000000000000000000000000000000000000000000...  0.536295
2       8192  0000000000000000000000000000000000000000000000...  0.517069
0          0  0000000000000000000000000000000000000000000000...  0.510910
0:00:00.562119
     test_key                                           bin_repr         0
39      18176  0000000000000000000000000000000000000000000000...  0.607865
135     50944  0000000000000000000000000000000000000000000000...  0.569914
23       9984  0000000000000000000000000000000000000000000000...  0.563210
7        1792  0000000000000000000000000000000000000000000000...  0.558687
103     34560  0000000000000000000000000000000000000000000000...  0.557672
0:00:07.776665
    test_key                                           bin_repr         0
47

In [82]:
#{'likely_keys': [291667320677550067, 291667320677484531, 291667320677418995, 291667320677353459, 291667320677812211], 'current_bits_guessed': 64}
print(keeloq_decrypt(dummy_ciphertexts[0],291667320677550067))

556861677


In [85]:
def algorithm1_pd(key_length_guess, n, ciphertexts , power_traces , prev_keys = [0] , current_bits_guessed = 0 ):
    #use pandas 
    
    #put into normal form
    temp_1 = [ key_register(x) for x in  prev_keys]  
    #add more MSB for guessing 
    temp_2 = [  ( (y <<current_bits_guessed) ^ x) for x in temp_1 for y in range(2**key_length_guess)]
    #put back into key register form for decrypting
    keyhyp = [ key_register(x) for x in  temp_2 ]
    keyhyp_d = { (key, ciphertext) : {} for key in keyhyp for ciphertext in ciphertexts}

    key_depth = current_bits_guessed + key_length_guess

    key_tests = pd.DataFrame([{ 'test_key':key, 'ciphertext':ciphertext}  for key in keyhyp for ciphertext in ciphertexts], dtype = 'uint64')
    
    #put into binary representaion (string) for easier inspection
    #key_tests['bin_repr'] = key_tests['test_key'].apply( lambda x : bin(x)[2:].zfill(64)  )
    
    # get the power trace
    key_tests['power_trace'] = key_tests['ciphertext'].map(lambda ct: power_traces[ct]['powertrace_sim_avg'][-key_depth] if ct in power_traces else None)
    #get the hypothetical power
    key_tests['dummy_power'] =  key_tests.apply(
        lambda row:   partial_decrypt_power_array(int(row['ciphertext']), int(row['test_key']) ,key_depth)[0] , axis=1)    

    correlation_by_group = key_tests.groupby(['test_key']).apply(lambda g: g['dummy_power'].corr(g['power_trace'])).reset_index()
    sorted_corr = correlation_by_group.sort_values(by=0,ascending=False)
    print(sorted_corr[:1]) 
    
    return({"likely_keys" : list(sorted_corr['test_key'][0:n]), "current_bits_guessed" : key_depth }) #Get the n most likely( highest max correlation) key guesses
    
for k,v in ciphertexts_d.items():
    v['powertrace_sim_avg'] = avg3(v['powertrace_sim'])[:-3]

start_time =  datetime.datetime.now()

r = algorithm1_pd(8, 10, dummy_ciphertexts  , ciphertexts_d, [0] , 0   )
print(datetime.datetime.now() - start_time)
print(r)

    test_key         0
71     18176  0.607865
0:00:20.581200
{'likely_keys': [18176, 50944, 9984, 1792, 34560, 26368, 42752, 24320, 30464, 22272], 'current_bits_guessed': 8}


In [ ]:
start_time =  datetime.datetime.now()
bits = 4 

#v['powertrace_sim_avg'] = (v['powertrace_sim'])[2::3][:-3]
#v['powertrace_sim_avg'] = avg3(v['powertrace_sim'])[:-2]

for k,v in ciphertexts_d.items():
    v['powertrace_sim_avg'] = avg3(v['powertrace_sim'])[:-3]

results = {"likely_keys" : [0], "current_bits_guessed" : 0 }
    
for i in range(64//bits):
    inner_start_time =  datetime.datetime.now()
    results = algorithm1_pd(4, 10, dummy_ciphertexts , ciphertexts_d, results["likely_keys"] , results["current_bits_guessed"]   )
    print(datetime.datetime.now() - inner_start_time)
    #print([bin(x)[2:].zfill(64)   for x in results["likely_keys"]])
    
print(datetime.datetime.now() - start_time)
print(results)
    


   test_key         0
4     16384  0.621136
0:00:01.212209
    test_key         0
39     18176  0.607865
0:00:14.078131
    test_key         0
47     18416  0.626686
0:00:17.152265
    test_key         0
67     18419  0.618805
0:00:17.766499
   test_key         0
4     18419  0.615545
0:00:20.789134
   test_key         0
4     18419  0.477047
0:00:25.557720
   test_key         0
6     18431  0.541043
0:00:30.004469
            test_key         0
42  3940649673967603  0.431445
0:00:31.531262
             test_key         0
147  3993426232100851  0.523841
0:00:32.368081
            test_key         0
81  3431575790307315  0.359042
0:00:37.215283
            test_key         0
66  3431713229260787  0.400185
0:00:37.969671
            test_key         0
35  3433925137418227  0.266609
0:00:41.245779
             test_key         0
112  3995492111370227  0.364963
0:00:43.812808
            test_key         0
77  3995495550699507  0.255599
0:00:46.693696
           test_key         0
1  39954